# **Simulações**


In [ ]:
# Bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import joblib

# Verificando quais alunos evadiram
def verifica_evadidos(row):
    if pd.isna(row['PONTO_VIRADA_2022']):  # Se PONTO_VIRADA_2022 for NaN
        return True  # O aluno evadiu
    elif pd.isna(row['PONTO_VIRADA_2020']):  # Se PONTO_VIRADA_2020 for NaN
        return False  # O aluno não evadiu
    elif pd.isna(row['PONTO_VIRADA_2021']):  # Se PONTO_VIRADA_2021 for NaN
        return True  # O aluno evadiu
    else:
        return False  # O aluno não evadiu

# Função para identificar o último ano de ponto de virada antes de NaN
def ultimo_ano(row):
    if row['EVADIU']:  # Verifica se EVADIU é verdadeiro
        if row['PONTO_VIRADA_2020'] in ['Sim', 'Não'] and pd.isna(row['PONTO_VIRADA_2021']):
            return 2020
        elif row['PONTO_VIRADA_2021'] in ['Sim', 'Não'] and pd.isna(row['PONTO_VIRADA_2022']):
            return 2021
        elif row['PONTO_VIRADA_2022'] in ['Sim', 'Não']:
            return 2022
    return pd.NA

def _generate_df_evasao(df):
    indicacao_evasao = ['NOME', 'PONTO_VIRADA_2020', 'PONTO_VIRADA_2021', 'PONTO_VIRADA_2022']
    df_evasao = df[indicacao_evasao]
    df_evasao['EVADIU'] = df_evasao.apply(verifica_evadidos, axis=1)
    # Aplicar a função para cada linha do DataFrame
    df_evasao['ULTIMO_ANO'] = df_evasao.apply(ultimo_ano, axis=1)
    return df_evasao

def _df_passos_magicos():
    df = pd.read_csv('data/raw/PEDE_PASSOS_DATASET_FIAP.csv', sep=';')
    df_evasao = _generate_df_evasao(df)
    # Mesclar os DataFrames com base na coluna 'NOME'
    df = df.merge(df_evasao[['NOME', 'EVADIU', 'ULTIMO_ANO']], on='NOME', how='left')
    # removendo linha problematica
    df = df.loc[~(df['INDE_2020'] == 'D980')]
    # substituindo null estranho
    df.loc[(df['INDE_2021'] == '#NULO!'),['INDE_2021']] = np.nan

    return df

def _get_saresp():
    return pd.read_csv('../data/processed/saresp.csv')

def _rename(df, cols_not_rename=['YEAR'], posfixo='_Y-1'):
    for col in cols_not_rename:
        df[col] = df[col].astype(str)
    for col in list(set(df.columns) - set(cols_not_rename)):
        df = df.rename(columns={col:col+posfixo})
    return df

def _get_xgb_model(path='models/xgb_model.pkl'):
    return joblib.load(path)


In [ ]:
model = _get_xgb_model(path='../models/xgb_model.pkl')

In [ ]:
def _test_df_cols(df):
    cols_in_last_year =  ['IDADE_Y-1', 'IAA_Y-1', 'IAN_Y-1', 'IDA_Y-1', 'IEG_Y-1', 'INDE_Y-1', 'IPP_Y-1', 'IPS_Y-1', 'IPV_Y-1']
    cols_in_current_year =  ['IDADE_Y', 'IAA_Y', 'IAN_Y', 'IDA_Y', 'IEG_Y', 'INDE_Y', 'IPP_Y', 'IPS_Y', 'IPV_Y']
    cols_identity = ['YEAR','NOME']
    cols_diff = set(cols_in_current_year + cols_in_last_year + cols_identity).difference(df.columns)
    return {
          'cols_diff':cols_diff
        , 'status_ok':True if len(cols_diff)<1 else False
        , 'df':df
    }

In [ ]:
def _load_new_data(file_path):
    df = pd.read_excel(file_path)
    cols_in =  ['IDADE', 'IAA', 'IAN', 'IDA', 'IEG', 'INDE', 'IPP', 'IPS', 'IPV']
    if not(_test_df_cols(df)['status_ok']):
        return _test_df_cols(df)
    
    # add dados da saresp em Y-1
    df_saresp = _get_saresp()
    df_saresp_m1 = _rename(df_saresp, posfixo='_Y-1')
    for col in ['YEAR','IDADE_Y-1']:
        df_saresp_m1[col] = df_saresp_m1[col].astype(int)
        df[col] = df[col].astype(int)
    df = df.merge(
        df_saresp_m1
        , on=['YEAR','IDADE_Y-1']
        )
    
    # saresp no ano corrente
    df_saresp_m = _rename(df_saresp, posfixo='_Y')
    for col in ['YEAR','IDADE_Y']:
        df_saresp_m[col] = df_saresp_m[col].astype(int)
        df[col] = df[col].astype(int)
    df = df.merge(
        df_saresp_m
        , on=['YEAR','IDADE_Y']
        )
    return {
          'cols_diff':[]
        , 'status_ok':True
        , 'df':df
    }


In [ ]:
response_new_data = _load_new_data('../data/processed/dado_simulacao.xlsx')
if (response_new_data['status_ok']):
    df_new_data = response_new_data['df']
else:
    print(response_new_data)

explainer dashboard

In [ ]:
df_new_data.index = df_new_data['NOME']
df_new_data = df_new_data.drop(columns={'NOME'})

In [ ]:
# https://explainerdashboard.readthedocs.io/en/latest/tabs.html

from explainerdashboard.custom import (
    ExplainerComponent, dbc, html,
    ShapDependenceComponent,
    ShapContributionsGraphComponent,
    ShapInteractionsComposite,
    ClassifierPredictionSummaryComponent,
    PdpComponent,
    ClassifierRandomIndexComponent,
    PosLabelSelector,
    IndexConnector, 
)
from explainerdashboard import ClassifierExplainer, RegressionExplainer
from explainerdashboard import ExplainerDashboard

class CustomDashboard(ExplainerComponent):
    def __init__(self, explainer, **kwargs):
        super().__init__(explainer, title="Previsões e Variáveis Decisórias")
        self.pos_label = 1
        self.index = ClassifierRandomIndexComponent(explainer,
                                                    hide_title=True, hide_index=False,
                                                    hide_slider=True, hide_labels=True,
                                                    hide_pred_or_perc=True,
                                                    hide_selector=True, hide_button=False)
        self.dependence = ShapDependenceComponent(explainer,
                            hide_selector=True, hide_percentage=True, hide_index=True,  
                            description="""
                                Este gráfico mostra a relação entre as variáveis e seu valor shap, 
                                fazendo com que consigamos investigar o relacionamento entre o valor das features e 
                                o impacto na previsão. Você pode checar se o modelo usa as features de acordo com o
                                esperado, ou usar o mesmo para aprender as relações que o modelo aprendeu entre
                                as features de entrada e a saída predita
                            """ ,
                            title="Dependência Shap", subtitle="Relação entre o valor da variável e o valor SHAP. Em cinza, o NOME selecionado!",             
                            cutoff=0.75, **kwargs)
        self.pdp = PdpComponent(explainer,
                            hide_selector=True, hide_cats=True, hide_index=True,
                            hide_depth=True, hide_sort=True,
                            description="""
                            O gráfico de dependência parcial (pdp) mostra como seria a previsão do modelo
                            se você alterar um recurso específico. O gráfico mostra uma amostra
                            de observações e como essas observações mudariam com as mudanças aplicadas
                            (linhas de grade). O efeito médio é mostrado em cinza. O efeito
                            de alterar o recurso para um único Nome é
                            mostrado em azul. Você pode ajustar quantas observações serão utilizadas para o
                            calculo, quantas linhas de grade mostrar e quantos pontos ao longo do
                            eixo x para calcular as previsões do modelo.
                            """,
                            title="Gráfico de Dependência Parcial",
                            subtitle="Como a previsão mudará se você mudar uma variável?",
                            **kwargs)
        self.ind_preds = ShapContributionsGraphComponent(explainer,
                            hide_selector=True, hide_cats=True, hide_index=True,
                            hide_depth=True, hide_sort=True,
                            description="""
                            Este gráfico mostra a contribuição que cada característica individual tem 
                            na previsão de uma observação específica. 
                            As contribuições (a partir da média da população) somam-se à previsão final.
                            Isso permite explicar exatamente como cada previsão individual foi construída 
                            a partir de todos os ingredientes individuais do modelo.
                            """,
                            title="Gráfico das Contribuições", subtitle="Como cada Variável contribuiu para a previsão?",
                            **kwargs)
        self.class_preds = ClassifierPredictionSummaryComponent(explainer,
                            hide_selector=True, hide_cats=True, hide_index=True,
                            hide_depth=True, hide_sort=True,
                            description="Mostra a probabilidade predita para cada situação, sendo False para não-evasão e True para Evasão.",
                            title="Previsões & Probabilidades",
                            **kwargs)
        self.connector = IndexConnector(self.index, [self.dependence, self.pdp, self.ind_preds, self.class_preds])

    def layout(self):
        return dbc.Container([
            dbc.Row([
                dbc.Col([
                    html.H3("Selecione um Aluno:"),
                    self.index.layout()
                ])
            ]),
            dbc.Row([
                dbc.Col([
                    self.ind_preds.layout(),
                ]),
                dbc.Col([
                    self.class_preds.layout(),
                ]),
            ]),
            dbc.Row([
                dbc.Col([
                    self.dependence.layout(),
                ]),
                dbc.Col([
                    self.pdp.layout(),
                ]),
            ])
            
        ])

In [ ]:
import dash
app = dash.Dash(__name__)
server = app.server
app.scripts.config.serve_locally=False
app.css.config.serve_locally=False

explainer = ClassifierExplainer(model, df_new_data, model.predict(df_new_data))
exp_dash = ExplainerDashboard(explainer, 
                            CustomDashboard, mode='inline',
                            server=server, url_base_pathname="/explainer_dashboard/",
                            header_hide_selector=True, port=8000, host='0.0.0.0',
                            description = """Esta área do dashboard mostra o funcionamento do modelo, explicando como ele realizou as suas predições""")
exp_dash.run()
# app.run()

In [ ]:
ExplainerDashboard.terminate(8000)

In [ ]:
print(CustomDashboard(explainer).index.description)

In [ ]:
index_ = CustomDashboard(explainer).connector

In [ ]:
index_.get_state_args(state_dict={'NOME'})

In [ ]:
exp_dash.app.+

In [ ]:
# exp_dash.terminate(8050)